<a href="https://colab.research.google.com/github/li-chu-hao/bioinfo/blob/main/from_RNA_to_protein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bioinformatics experiment: From RNA to protein

## Introduction

**实验目的：**

通过完成核苷酸序列的拼接、翻译、折叠，对生物信息学有进一步的认识。

**实验步骤：**

0. 运行你的第一个python程序。
1. 对RNA进行模拟测序，生成核苷酸序列片段。
2. 序列拼接
3. 翻译为氨基酸
4. 氨基酸折叠


## Step 0：运行你的第一个python程序：

In [ ]:
print("\n--------------")
print("Hello world!")
print("Your python environment has been successfully set up! ")
print("--------------")

## Step 1: 对RNA进行模拟测序，生成核苷酸序列片段。

1.1 下载血红蛋白亚基β的mRNA序列，保存到`rna.fna`文件。

In [ ]:
!wget -O "rna.fna" "https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=fasta_cds_na&id=1401724401&extrafeat=null&conwithfeat=on&hide-cdd=on&ncbi_phid=CE89C6D35DF019A1000000000AD308E1"

1.2 查看`rna.fna`文件内容

In [ ]:
!cat rna.fna

1.3 对RNA进行模拟测序，生成核苷酸序列片段。

安装依赖库：

In [ ]:
!python3 -m pip install biopython

生成reads：

In [ ]:
# @title 生成reads代码
# 1. 读取序列。
# 2. 根据reads长度、覆盖度生成随机数。
# 3. 把生成的随机n数作为序列的起点，n+read length作为终点，contigs.
import random
from Bio import SeqIO

seq = SeqIO.read("rna.fna", "fasta")

contig_len = 50
coverage = 50

print("\n序列长度：", len(seq.seq))
contig_num = int(len(seq.seq)*coverage / contig_len)

print("生成read的长度：", contig_len)
print("生成read的覆盖度（平均每个碱基被测序到的次数）：", coverage)
print("预计生成的reads条数：", contig_num)

start = 0
end = len(seq.seq) - contig_len

with open("01_output_reads.txt", "w") as f:
    start_points = [random.randint(start, end) for i in range(contig_num)]
    for n in start_points:
        print(str(seq.seq[n:n+contig_len]), file=f)

print("\n运行完毕！请检查结果文件'01_output_reads.txt'。")


查看生成的reads文件的前10行：

In [ ]:
!head 01_output_reads.txt

## Step 2: 序列拼接

In [ ]:
# @title 序列拼接代码
# 这里是正确的代码块顺序：A-C-E-D-B


#####################Code block A: 读取传入参数。########################
# 代码注释：min_overlap 参数默认为25，如果用户在命令行末端添加了
# 一个数字，min_overlap将被设置为该数值。
import sys
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
min_overlap = 25
if len(sys.argv) == 2:
    min_overlap = int(sys.argv[1])
print("min_overlap被设置为：", min_overlap)
#####################Code block A: 读取传入参数。########################



#####################Code block C: 定义函数。########################
# 代码注释：重复运行的代码，为了避免写很多次，我们会定义为“函数”。
# 以下“merge_contig”函数的作用是检测两条序列是否有至少“min_overlap”
# 个碱基的重叠，如果是，那就合并到一起。
def merge_contig(a, b, min_overlap=25):
    min_L = min(len(a), len(b))
    # 处理相互包含的
    if a in b:
        return b
    if b in a:
        return a
    # 处理头尾相接的
    # a head overlap with b tail
    for i in range(min_overlap, min_L+1):
        if a[:i] == b[-i:]:
            c = b + a[i:]
            return c
    # a tail overlap with b head
    for i in range(min_overlap, min_L+1):
        if b[:i] == a[-i:]:
            c = a + b[i:]
            return c
    return None
#####################Code block C: 定义函数。########################



#####################Code block E: 读取reads。########################
# 代码注释：读取所有生成的reads。
reads = open("01_output_reads.txt").read().splitlines()
#####################Code block E: 读取reads。########################



#####################Code block D: 执行拼接。########################
# 代码注释：对reads进行两两比较，如果有重叠，就合并在一起，直到只剩下
# 一条read，或者reads的数目不再变化。
n_reads = len(reads)
while len(reads) > 1:
    for i in range(len(reads)):
        for j in range(i+1, len(reads)):
            a = reads[i]
            b = reads[j]
            if a and b:
                c = merge_contig(a, b, min_overlap=min_overlap)
                if c:
                    reads[i] = c
                    reads[j] = None
    reads = [i for i in reads if i]
    n_reads_tmp = len(reads)
    reduced_reads = n_reads - n_reads_tmp
    if reduced_reads == 0:
        print("reads数没有发生改变，这意味着无法拼成单条完整的序列。退出程序。")
        exit()
    n_reads = n_reads_tmp
#####################Code block D: 执行拼接。########################



#####################Code block B: 输出结果。########################
# 代码注释：把得到的结果输出到文件。
with open("02_OLC_assembly.txt", "w") as f:
    record = SeqRecord(
        Seq(reads[0]),
        id="HBB_assembly",
        description="HBB assembly by OLC",
)
    SeqIO.write(record, f, "fasta")

print("\n运行完毕！请检查'02_OLC_assembly.txt'文件。")
#####################Code block B: 输出结果。########################



查看结果文件：

In [ ]:
!cat 02_OLC_assembly.txt

## Step 3: 翻译为氨基酸

In [ ]:
from Bio import SeqIO
seq = SeqIO.read("02_OLC_assembly.txt", "fasta")
seq.seq = seq.seq.translate()
seq.id = "HBB_amino_sequence"
seq.description = "HBB amino sequence"
with open("translated.txt", "w") as f:
    SeqIO.write(seq, f, "fasta")

print("RNA序列翻译为氨基酸序列，运行完毕！")

In [ ]:
!cat translated.txt

## Step 4: 氨基酸折叠

打开上一步生成的`translated.txt`文件，把其中的氨基酸序列复制到<a href="https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb" target="_blank">colabfold网站</a>中的`query_sequence`处，然后点击上方菜单栏中的“代码执行程序，然后点击“全部运行”。模拟折叠该氨基酸序列大概需要5-7分钟。

理解蛋白折叠和alphafold。

运行完毕后，会自动下载一个压缩包，打开该压缩包，可以看到5个以".pdb"结尾的文件，分别对应5种折叠方案。把该文件提交到<a href="https://www.rcsb.org/3d-view" target="_blank">蛋白三维结构查看网站</a>，可以查看该氨基酸的三维结构。

探索：修改部分氨基酸序列，看看折叠的结果有什么不一样。